<a href="https://colab.research.google.com/github/girija2204/DLTopics/blob/master/PyTorch/Convolutional_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import torch
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary

In [ ]:
train_data=datasets.MNIST(root='dataset/',train=True,transform=transforms.ToTensor(),download=True)
test_data=datasets.MNIST(root='dataset/',train=False,transform=transforms.ToTensor(),download=True)
train_loader=DataLoader(dataset=train_data,batch_size=128,shuffle=True)
test_loader=DataLoader(dataset=test_data,batch_size=128,shuffle=True)

In [3]:
DEVICE=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
learning_rate=0.001
epochs=20

In [20]:
class CNN(nn.Module):
  def __init__(self,in_channels=1,num_classes=10):
    super(CNN,self).__init__()
    self.conv1=nn.Conv2d(in_channels=in_channels,out_channels=8,kernel_size=(3,3),stride=(1,1),padding=(1,1))
    self.pool1=nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
    self.conv2=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1))
    self.fc1=nn.Linear(16*7*7,num_classes)

  def forward(self,x):
    x=F.relu(self.conv1(x))
    x=self.pool1(x)
    x=F.relu(self.conv2(x))
    x=self.pool1(x)
    x=x.reshape(x.shape[0],-1)
    x=self.fc1(x)
    return x

In [28]:
model=CNN().to(DEVICE)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [29]:
for epoch in range(epochs):
  for batch_idx, (data, target) in enumerate(train_loader):
    data=data.to(DEVICE)
    target=target.to(DEVICE)
    scores=model(data)
    loss=criterion(scores,target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [32]:
def check_accuracy(loader,model):
  if loader.dataset.train:
    print(f"Checking accuracy for the train data")
  else:
    print(f"Checking accuracy for the test data")

  num_correct = 0
  num_total = 0
  model.eval()
  
  with torch.no_grad():
    for data,target in loader:
      data=data.to(DEVICE)
      target=target.to(DEVICE)
      scores=model(data)
      _,predictions=scores.max(1)
      num_correct+=(predictions==target).sum()
      num_total+=predictions.shape[0]
  print(f"Accuracy: {(num_correct/num_total)*100:2f}")
  model.train()

In [33]:
check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

Checking accuracy for the train data
Accuracy: 99.446663
Checking accuracy for the test data
Accuracy: 98.709991
